<a href="https://colab.research.google.com/github/LuViBeBe93/tesis/blob/main/GPT2_F3_V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install transformers pandas torch bert-score nltk tabulate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 4.3 MB/s eta 0:00:00


In [2]:
# Importación de bibliotecas
from google.colab import files
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from transformers import AutoTokenizer, AutoModelWithLMHead, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from bert_score import score
import math
import nltk
nltk.download('punkt')
import numpy as np

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [3]:
# Parámetros generales del modelo y configuración
PARAMETROS_MODELO = {
    "modelo_preentrenado": "datificate/gpt2-small-spanish",
    "max_length": 128,
    "num_train_epochs": 7,  # Optimizado con bayesiana
    "batch_size_entrenamiento": 4,  # Optimizado con bayesiana
    "num_beams": 3,  # Optimizado con bayesiana
    "no_repeat_ngram_size": 3,  # Optimizado con bayesiana
    "temperature_values": [0.5, 0.7, 1.0, 1.2, 1.5],  # Rango de búsqueda para temperature
    "top_p_values": [0.5, 0.6, 0.7, 0.8, 0.9],  # Rango de búsqueda para top_p
    "logging_steps": 100,
    "save_steps": 500,
    "save_total_limit": 2,
    "evaluation_strategy": "epoch",
    "output_dir": "./gpt2_noticias_model",
    "logging_dir": "./logs",
    "pad_token_id": None,  # Se asigna automáticamente al id del token EOS si es None
}


In [4]:
# Configuración para usar GPU si está disponible
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
# Subir el archivo de datos
uploaded = files.upload()
data = pd.read_excel('BASE_30122024.xlsx')
data = pd.DataFrame(data)

Saving BASE_30122024.xlsx to BASE_30122024.xlsx


In [6]:
# Convertir datos numéricos a texto
def datos_a_texto(row):
    texto_datos = (
        f"Año: {row.get('Año', 'NaN')}, Mes: {row.get('Mes', 'NaN')}, "
        f"Trimestre: {row.get('Trimestre', 'NaN')}, Frecuencia: {row.get('Frecuencia', 'NaN')}, "
        f"Año_comparación: {row.get('Año_comparación', 'NaN')}, País: {row.get('País', 'NaN')}, "
        f"Tendencia: {row.get('Tendencia', 'NaN')}, Sector: {row.get('Sector', 'NaN')}, "
        f"Indicador: {row.get('Indicador', 'NaN')}, Valor_actual: {row.get('Valor_actual', 'NaN')}, "
        f"Variación: {row.get('Variación', 'NaN')}, Valor_comparación: {row.get('Valor_comparación', 'NaN')}, "
        f"Entidad: {row.get('Entidad', 'NaN')}"
    )
    return texto_datos

# Crear una columna con los datos numéricos convertidos a texto
data["entrada_texto"] = data.apply(datos_a_texto, axis=1)
data["texto_final"] = data["entrada_texto"] + " </s> Noticia: " + data["Noticia"]

In [7]:
# Dataset personalizado
class NoticiasDataset(Dataset):
    def __init__(self, data, tokenizer, max_length):
        self.data = data
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        texto = self.data.iloc[idx]["texto_final"]
        encoding = self.tokenizer(
            texto,
            truncation=True,
            padding="max_length",
            max_length=self.max_length,
            return_tensors="pt",
        )
        input_ids = encoding["input_ids"].flatten()
        attention_mask = encoding["attention_mask"].flatten()

        return {
            "input_ids": input_ids,
            "attention_mask": attention_mask,
            "labels": input_ids,  # En GPT, las etiquetas son las mismas que la entrada
        }


In [8]:
# Dividir datos en 80% entrenamiento y 20% evaluación
train_data, eval_data = train_test_split(data, test_size=0.2, random_state=42)


In [9]:
# Inicializar el tokenizador y modelo GPT-2
tokenizer = AutoTokenizer.from_pretrained(PARAMETROS_MODELO["modelo_preentrenado"])
model = AutoModelWithLMHead.from_pretrained(PARAMETROS_MODELO["modelo_preentrenado"])

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/620 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/817 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/850k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/508k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/387 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/modeling_auto.py:1838: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/510M [00:00<?, ?B/s]

In [10]:
# Ajustar el tokenizador para manejar padding
tokenizer.pad_token = tokenizer.eos_token
model.resize_token_embeddings(len(tokenizer))

Embedding(50257, 768)

In [11]:
# Crear datasets y loaders
train_dataset = NoticiasDataset(train_data, tokenizer, PARAMETROS_MODELO["max_length"])
eval_dataset = NoticiasDataset(eval_data, tokenizer, PARAMETROS_MODELO["max_length"])

# Crear DataLoader para evaluación
eval_dataloader = DataLoader(eval_dataset, batch_size=PARAMETROS_MODELO["batch_size_entrenamiento"])

In [12]:
# Configuración del entrenamiento
training_args = TrainingArguments(
    output_dir=PARAMETROS_MODELO["output_dir"],
    num_train_epochs=PARAMETROS_MODELO["num_train_epochs"],
    per_device_train_batch_size=PARAMETROS_MODELO["batch_size_entrenamiento"],
    save_steps=PARAMETROS_MODELO["save_steps"],
    save_total_limit=PARAMETROS_MODELO["save_total_limit"],
    evaluation_strategy=PARAMETROS_MODELO["evaluation_strategy"],
    logging_dir=PARAMETROS_MODELO["logging_dir"],
    logging_steps=PARAMETROS_MODELO["logging_steps"],
)

# Crear el Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
)

# Entrenar el modelo
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss
1,0.326800,0.269236
2,0.258000,0.230041
3,0.230900,0.216880
4,0.210200,0.203514
5,0.203400,0.199466
6,0.190800,0.195016
7,0.185500,0.192351


TrainOutput(global_step=4466, training_loss=0.2635181626022209, metrics={'train_runtime': 1068.8244, 'train_samples_per_second': 16.701, 'train_steps_per_second': 4.178, 'total_flos': 1166015692800000.0, 'train_loss': 0.2635181626022209, 'epoch': 7.0})

In [13]:
# Función para generar noticias
def generar_noticia(model, tokenizer, datos, temperature, top_p):
    entrada_texto = datos_a_texto(datos)
    inputs = tokenizer(
        entrada_texto,
        return_tensors="pt",
        truncation=True,
        padding=True,
        max_length=PARAMETROS_MODELO["max_length"],
    ).to(model.device)

    input_ids = inputs["input_ids"]
    attention_mask = inputs["attention_mask"]

    # Configurar pad_token_id solo si no está configurado
    if model.config.pad_token_id is None:
        model.config.pad_token_id = tokenizer.eos_token_id

    # Generar texto
    outputs = model.generate(
        input_ids=input_ids,
        attention_mask=attention_mask,
        max_length=PARAMETROS_MODELO["max_length"],
        num_beams=PARAMETROS_MODELO["num_beams"],
        no_repeat_ngram_size=PARAMETROS_MODELO["no_repeat_ngram_size"],
        early_stopping=True,
        temperature=temperature,
        top_p=top_p,
        do_sample=True,
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

In [14]:
# Evaluación de parámetros
def calcular_perplejidad(model, tokenizer, eval_dataloader):
    model.eval()
    total_loss = 0
    total_tokens = 0

    for batch in eval_dataloader:
        input_ids = batch["input_ids"].to(model.device)
        attention_mask = batch["attention_mask"].to(model.device)
        labels = batch["labels"].to(model.device)

        with torch.no_grad():
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            total_loss += loss.item() * input_ids.size(0)
            total_tokens += input_ids.size(0)

    return math.exp(total_loss / total_tokens)

def evaluar_parametros(model, tokenizer, eval_data, temperature, top_p):
    generadas = []
    referencias = []

    for i in range(len(eval_data)):
        datos = eval_data.iloc[i]
        referencias.append(datos["Noticia"])
        generadas.append(generar_noticia(model, tokenizer, datos, temperature, top_p))

    # Calcular métricas
    P, R, F1 = score(generadas, referencias, lang="es", model_type="bert-base-uncased")
    bleu_scores = [
        sentence_bleu([ref.split()], gen.split()) for ref, gen in zip(referencias, generadas)
    ]
    perplejidad = calcular_perplejidad(model, tokenizer, eval_dataloader)

    return {
        "temperature": temperature,
        "top_p": top_p,
        "bert_f1": F1.mean().item(),
        "bert_Precision": P.mean().item(),
        "bert_Recall": R.mean().item(),
        "bleu": np.mean(bleu_scores),
        "perplejidad": perplejidad
    }

# Evaluar combinaciones de parámetros
resultados = []
for temp in PARAMETROS_MODELO["temperature_values"]:
    for tp in PARAMETROS_MODELO["top_p_values"]:
        print(f"Evaluando temperature={temp}, top_p={tp}...")
        resultado = evaluar_parametros(model, tokenizer, eval_data, temp, tp)
        resultados.append(resultado)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Evaluando temperature=0.5, top_p=0.5...


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

...

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

Evaluando temperature=1.5, top_p=0.8...


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

Evaluando temperature=1.5, top_p=0.9...


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end gene

In [15]:
# Crear la tabla de resultados
tabla_resultados = pd.DataFrame(resultados)
print("\nResultados completos:")
print(tabla_resultados)


Resultados completos:
    temperature  top_p   bert_f1  bert_Precision  bert_Recall      bleu  \
0           0.5    0.5  0.646060        0.550258     0.785812  0.143650   
1           0.5    0.6  0.646439        0.550633     0.786174  0.144440   
2           0.5    0.7  0.646498        0.550544     0.786560  0.143404   
3           0.5    0.8  0.646576        0.550870     0.786119  0.143913   
4           0.5    0.9  0.646549        0.550571     0.786749  0.145101   
5           0.7    0.5  0.646494        0.550455     0.786762  0.143719   
6           0.7    0.6  0.646288        0.550505     0.785994  0.143606   
7           0.7    0.7  0.646454        0.550486     0.786586  0.143029   
8           0.7    0.8  0.646617        0.550699     0.786658  0.145313   
9           0.7    0.9  0.646899        0.551084     0.786642  0.146405   
10          1.0    0.5  0.646296        0.550458     0.786112  0.144454   
11          1.0    0.6  0.646545        0.550651     0.786488  0.144441   
12